<a href="https://colab.research.google.com/github/amoghjoshi0098/low-light/blob/main/Unet_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rawpy
import argparse
import rawpy

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.autograd import Variable
import os
from tqdm import tqdm
import cv2
import numpy as np
import pandas as pd
import os

     |████████████████████████████████| 2.2 MB 5.1 MB/s 


In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument("--continue_train",help = "True if u continue trainig, False if u want to train from starting", default=False)
# parser.add_argument("--epoch_no",help="epoch number to load checkpoint",default = 10)
# parser.add_argument("--lr",help = 'learning_rate',default=0.01)
# parser.add_argument("--v",help = 'result_folder_version')
# args = parser.parse_args()
# if args.continue_train:
#     load_model = np.int(args.continue_train)
# else:
#     load_model = 0
# if args.epoch_no:
#     load_model_no = args.epoch_no
# if args.lr:
#     learning_rate = np.float32(args.lr)
# if args.v:
#     version = args.v
load_model = 0

In [ ]:
no_epochs = 1000
batch_size = 8
save_freq = 5
load_epoch = 0
result_path = "/content/drive/MyDrive/Sonymodels"
if(os.path.exists(result_path) == False):
    os.mkdir(result_path)
if(os.path.exists(result_path + 'models/') == False):
    os.mkdir(result_path + 'models/')
if(os.path.exists(result_path + 'image_mask/') == False):
    os.mkdir(result_path + 'image_mask/')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [ ]:
class DataSet(torch.utils.data.Dataset):
  def __init__(self, csv_file, root_directory_ground_truth,root_directory_input):
    #initialisations
    super(DataSet,self).__init__()
    self.annotations = pd.read_csv(csv_file)
    self.root_directory_input = root_directory_input
    self.root_directory_ground_truth = root_directory_ground_truth
  def __len__(self):
    return len(self.annotations)
  def __getitem__(self, index):
    temp = self.annotations.iloc[index,0]
    temp =  temp[2:len(temp)]
    image_path = os.path.join(self.root_directory_input, temp)
    temp = self.annotations.iloc[index,1]
    temp =  temp[2:len(temp)]
    image_mask_path = os.path.join(self.root_directory_ground_truth,temp)
    #print(image_path,image_mask_path)
    image = rawpy.imread(image_path)
    image_mask = rawpy.imread(image_mask_path)
    image = image.raw_image_visible
    image_mask = image_mask.raw_image_visible
    image = np.float32(image/255.0)
    image_mask = np.float32(image_mask/255.0)
    #print(np.max(image),np.max(image_mask))
    image = torch.from_numpy(image)
    image_mask = torch.from_numpy(image_mask)
    return (image,image_mask)



In [ ]:
root=os.getcwd()
os.chdir(root)

#Dataset
dataset = DataSet(csv_file = '/content/drive/MyDrive/Sony/Trial.csv',
    root_directory_input = '/content/drive/MyDrive/Sony',
    root_directory_ground_truth ='/content/drive/MyDrive/Sony')


#split
print('length of dataset = ',len(dataset))
train_data, test_data = torch.utils.data.random_split(dataset,[1500,364])

#Dataloader
train_load = torch.utils.data.DataLoader(train_data,batch_size = batch_size,shuffle = True,num_workers=4)
test_load = torch.utils.data.DataLoader(test_data,batch_size = batch_size,shuffle = True,num_workers=0)
save_load = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True, num_workers=0)
print('Data Loader initialized')


length of dataset =  1864
Data Loader initialized


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
#unet network
class convBlock(nn.Module):
  def __init__(self,in_ch,out_ch,kernel_size):
    super(convBlock,self).__init__()
    self.conv1 = nn.Conv2d(in_ch,out_ch,kernel_size,padding=1)
    self.conv2 = nn.Conv2d(out_ch,out_ch,kernel_size,padding=1)
    self.conv3 = nn.Conv2d(out_ch,out_ch,kernel_size)
    self.maxpooling = nn.MaxPool2d(2)
    self.relu = nn.ReLU(True)
  def forward(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    #x = self.conv3(x)
    x = self.maxpooling(x)
    x = self.relu(x)
    return x




In [ ]:
class decoder(nn.Module):
  def __init__(self, in_ch, out_ch , kernel_size):
    super(decoder, self ).__init__()
    self.conv1 = nn.Conv2d( in_ch,out_ch , kernel_size,padding=1)
    self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size,padding=1)
    self.upsamplenn=nn.Upsample( scale_factor=2, mode='bilinear',align_corners=True)
    self.relu= nn.ReLU(True)
  def forward(self,x):
   x = self.conv1(x)
   x = self.conv2(x)
   x = self.upsamplenn(x)
   x = self.relu(x)
   return x

In [ ]:
class unet(nn.Module):
  def __init__(self):
    super(unet,self).__init__()
    self.convblock1 = convBlock(1,64,3)
    self.convblock2 = convBlock(64,128,3)
    self.convblock3 = convBlock(128,256,3)
    self.convblock4 = convBlock(256,512,3)
    self.convlayer5 = nn.Conv2d(512,1024,3,padding = 1)
    self.convlayer6 = nn.Conv2d(1024,512,3,padding = 1)
    self.convblock7 = decoder(1024,256,3)
    self.convblock8 = decoder(512,128,3)
    self.convblock9 = decoder(256,64,3)
    self.convblock10 = decoder(128,64,3)
    self.convlayer11 = nn.Conv2d(64,3,3,padding = 1)

  def forward(self,x):
    print('x',x.shape)

    x1 = self.convblock1(x) 
    print('x1',x1.shape)

    x2 = self.convblock2(x1)
    print('x2',x2.shape)

    x3 = self.convblock3(x2)
    print('x3',x3.shape)

    x4 = self.convblock4(x3)
    print('x4',x4.shape)

    x5 = self.convlayer5(x4)
    print('x5',x5.shape)

    x6 = self.convlayer6(x5)
    print('x6',x6.shape)

    catX = torch.cat((x6,x4),1)
    print('cat',catX.shape)

    x7 = self.convblock7(catX)
    print('x7',x7.shape)

    catX = torch.cat((x7,x3),1)
    print('cat',catX.shape)

    x8 = self.convblock8(catX)
    print('x8',x8.shape)


    catX = torch.cat((x8,x2),1)
    print('cat',catX.shape)

    x9 = self.convblock9(catX)
    print('x9',x9.shape)
   
    catX = torch.cat((x9,x1),1)
    print('cat',catX.shape)

    x10 = self.convblock10(catX)
    print('x10',x10.shape)

    xOut = self.convlayer11(x10)
    print('xOut',xOut.shape)

    return xOut

model = unet()

In [ ]:
# input = torch.randn((1,1,512,512))
# output = model(input)

In [ ]:
#optimiser
#nn.MSELoss(size_average=None, reduce=None, reduction='mean')
criterion = nn.MSELoss()
optimiser = optim.Adam(model.parameters(),lr = 1e-3)

if load_model == 1:
    print("Loading Model")
    load_path = result_path + 'models/' + str(load_model_no) + "_model.pth"
    model.load_state_dict(torch.load(load_path)['model_state_dict'])
    load_epoch = torch.load(load_path)["epoch"]
    optimizer.load_state_dict(torch.load(load_path)['optimizer_state_dict'])
    optimizer.param_groups[0]['lr'] = learning_rate
    print("Weights intialised")
    print('Learning Rate = ', optimizer.param_groups[0]['lr'])
    f = open(result_path + 'Logs.txt','a')
    f.write('continue from' + str(load_epoch) + '\n')
    f.close()
    model.to(device)


In [ ]:
f = open(result_path + 'Logs.txt','a')
f.write('new_run \n')
f.close()
##Train the model
def train(train_load,epoch):
    model.train()
    current_loss = 0.0
    test_loss=0.0
    train_loss = 0.0
    loop=tqdm(enumerate(train_load),total = 5,leave=False)
    for i,data in loop:
        inputs,labels = data
        inputs,labels = inputs.to(device),labels.to(device)
        inputs=inputs.unsqueeze(0)
        inputs = inputs.permute(1, 1, 512, 512)
        outputs = model(inputs)
        outputs = outputs[:,0,:,:]
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        current_loss += loss.item()
        #print(inputs.shape,current_loss)
        optimizer.zero_grad()   
        break
    train_loss = current_loss/len(train_load)
    print('trainig_loss for ' + str(epoch+1) +' epoch = ' + str(train_loss))
    current_loss = 0.0
    f = open(result_path + 'Logs.txt','a')
    writer = 'training_loss for ' + str(epoch +1) + ' epoch = ' + str(train_loss)
    f.write(writer)
    f.close()
    return train_loss

In [ ]:
##Test The model
def test(test_load,epoch):
    model.eval()
    current_loss = 0.0
    loop = tqdm(enumerate(test_load),total=len(test_load),leave=False)
    for j,data in loop:
        inputs,labels = data
        inputs,labels = inputs.to(device),labels.to(device)
        ##inputs = inputs.permute(0,3,1,2)
        with torch.no_grad():
           outputs = model(inputs)
           outputs = outputs[:,0,:,:]
           loss = criterion(outputs,labels)
           current_loss += loss.item()
    test_loss = current_loss/len(test_load)
    current_loss = 0.0
    print('validation_loss for ' + str(epoch+1) + ' epoch  = ' + str(test_loss))
    f = open(result_path + 'Logs.txt','a')
    writer = 'validation_loss for ' + str(epoch + 1) + ' epoch = ' + str(test_loss)
    f.write(writer)
    f.close()
    return test_loss



In [ ]:
def save_result(test_load,epoch):
    model.train()
    model_path = result_path + 'models/' + str(epoch+1) + '_model.pth'
    image_path = result_path + 'image_mask/' + str(epoch+1) + '_mask.png'
    if(epoch%save_freq == 0):
        torch.save({'epoch' : epoch,
            'model_state_dict' : model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict()},
            model_path)
    loop = tqdm(enumerate(test_load),total=2,leave=False)
    for j,data in loop:
        inputs,labels = data
        inputs,labels = inputs.to(device),labels.to(device)
        ##inputs = inputs.permute(0,3,1,2)
        with torch.no_grad():
            outputs = model(inputs)
            outputs = outputs[0,0,:,:]
            outputs = np.array(outputs.cpu())
            cv2.imwrite(image_path, outputs)
            #print("Saved result of " + str(epoch + 1) + str(outputs.shape))


In [ ]:
Tr_loss = []
Te_loss = []
for epoch in range(load_epoch,no_epochs):
    temp = train(train_load,epoch)
    Tr_loss.append(temp)
    temp = test(test_load,epoch)
    Te_loss.append(temp)
    save_result(test_load,epoch)
    PATH_latest = result_path + 'latest.pth'
    Tr = pd.DataFrame(np.array(Tr_loss))
    Tr.to_csv(result_path + 'train_loss.csv',header = None, index = None)
    Te = pd.DataFrame(np.array(Te_loss))
    Te.to_csv(result_path + 'test_loss.csv',header = None, index = None)
    torch.save({'epoch' : epoch,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict':optimizer.state_dict()},
        result_path + 'latest_model.pth')
print('Finished Trianing')
PATH_final = result_path + 'Final_Model.pth'
torch.save(model.state_dict(),PATH_final)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


LibRawIOError: ignored